# Project UAS - Pengolahan Bahasa Alami
##  SIMILARITY DAN CLUSTERING PADA BERITA OLAHRAGA DI CNN

**Anggota Kelompok:**
1. Emmanuel Mathew Krisna Rata - 200710530
2. Antonius Indra Wardhana - 200710542
3. Farrel Rayner Nathanael Geovano - 200710560
4. Edmond Sorensen - 200710617
4. Paulus Bayu Kurnia Wicaksono - 200710691

## Objective
Mempermudah pembaca dalam menemukan berita yang dituju 

Meningkatkan efisiensi dan efektivitas dalam mengelola berita 

## Latar Belakang
POIN 1 

Klasifikasi berita olahraga juga dapat membantu pembaca untuk menemukan berita yang lebih relevan dengan minatnya. Hal ini karena berita-berita olahraga yang dikategorikan akan lebih mudah dicari dan diakses. 

POIN 2 

Klasifikasi berita olahraga dapat membantu media massa untuk mengelola berita olahraga dengan lebih efisien dan efektif. Hal ini dapat dilakukan dengan memisahkan berita-berita olahraga berdasarkan kategorinya masing-masing 

## Deskripsi Dataset
SUMBER 

Sumber berita online terkait olahraga diambil dari CNN 

KARAKTERISTIK 

Label yang dipakai adalah Judul berita, kategori berita. Jumlah data ada pada dataset adalah 2000 baris 

## Pengembangan Model Similarty Text


### Inisialisasi
Silakan isi bagian ini dengan mengimport library-library yang akan digunakan. 

In [54]:
import os
import re
import sys
import string
import modSpellChecker_1 as sc
from contractions_1 import CONTRACTION_MAP
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import numpy as np
import pandas as pd
import operator
from sklearn.model_selection import train_test_split
import time

In [55]:
dataset = pd.read_csv('cnn_sport.csv')
dataset

,title,category,link
0,Klasemen Liga Inggris: Tottenham Gusur Man Cit...,Sepakbola,https://www.cnnindonesia.com/olahraga/20231024...
1,Khamzat Disebut Tak Pantas Dapat Duel Gelar Ju...,Olahraga Lainnya,https://www.cnnindonesia.com/olahraga/20231023...
2,"Hasil Liga Inggris: Son Cetak Gol, Tottenham H...",Sepakbola,https://www.cnnindonesia.com/olahraga/20231024...
3,Binder Merasa Kasihan Martin Gagal Menang di M...,Moto GP,https://www.cnnindonesia.com/olahraga/20231024...
4,FOTO: Tottenham Jaga Rekor Belum Terkalahkan U...,Sepakbola,https://www.cnnindonesia.com/olahraga/20231024...
...,...,...,...
1995,Jadwal Siaran Langsung Final Hong Kong Open: 3...,Raket,https://www.cnnindonesia.com/olahraga/20230917...
1996,Klasemen Liga Inggris Usai MU Terpuruk di Old ...,Sepakbola,https://www.cnnindonesia.com/olahraga/20230917...
1997,Ronaldo Tunjukkan Gairah Besar di Saudi Pro Le...,Sepakbola,https://www.cnnindonesia.com/olahraga/20230917...
1998,"2 Pemain Al Raed Berebut Tendang Penalti, Rona...",Sepakbola,https://www.cnnindonesia.com/olahraga/20230917...


In [56]:
feature = dataset.iloc[:,0]
label = dataset.iloc[:,0]

In [57]:
label[0:10]

0    Klasemen Liga Inggris: Tottenham Gusur Man Cit...
1    Khamzat Disebut Tak Pantas Dapat Duel Gelar Ju...
2    Hasil Liga Inggris: Son Cetak Gol, Tottenham H...
3    Binder Merasa Kasihan Martin Gagal Menang di M...
4    FOTO: Tottenham Jaga Rekor Belum Terkalahkan U...
5    INFOGRAFIS: Jadwal Timnas Indonesia di Kualifi...
6          Messi Banggakan Capaian Bersama Inter Miami
7    Erick Thohir Soal Persiapan Piala Dunia U-17: ...
8    Bagnaia Mulai Pongah Jelang MotoGP Thailand da...
9    Sidang Komdis PSSI: Hugo Samir Pukul Pemain Pe...
Name: title, dtype: object

In [58]:
feature[0:10]

0    Klasemen Liga Inggris: Tottenham Gusur Man Cit...
1    Khamzat Disebut Tak Pantas Dapat Duel Gelar Ju...
2    Hasil Liga Inggris: Son Cetak Gol, Tottenham H...
3    Binder Merasa Kasihan Martin Gagal Menang di M...
4    FOTO: Tottenham Jaga Rekor Belum Terkalahkan U...
5    INFOGRAFIS: Jadwal Timnas Indonesia di Kualifi...
6          Messi Banggakan Capaian Bersama Inter Miami
7    Erick Thohir Soal Persiapan Piala Dunia U-17: ...
8    Bagnaia Mulai Pongah Jelang MotoGP Thailand da...
9    Sidang Komdis PSSI: Hugo Samir Pukul Pemain Pe...
Name: title, dtype: object

### Normalisasi Data
Silakan isi bagian ini dengan script untuk keperluan normalisasi data teks yang akan digunakan untuk pengembangan model klasifikasi teks. 
Untuk setiap strategi normalisasi yang akan digunakan sebaiknya dibuat fungsi tersendiri dan dipanggil dalam satu fungsi normalisasi. Jangan lupa pada bagian ini ditampilkan contoh hasil normalisasi datanya.

In [59]:
#fungsi2 untuk normalisasi dataset
character =['z','y','x','w','v','u','t','s','r','q','p','o','n','m','l','k','j','i','h','g','f','e','d','c','b','a',',','.',';',':','-','...','?','!','(',')','[',']','{','}','<','>','"','/','\'','#','-','@']
def repeatcharNormalize(text):
    for i in range(len(character)):
        charac_long = 5
        while charac_long>=2:
            char=character[i]*charac_long
            text=text.replace(char,character[i])
            charac_long-=1
        return text
def spellNormalize(text):
    spellCheck = []
    for i in text:
        if i not in character:
            j=sc.correction(i)
            spellCheck.append(j)
        else:
            spellCheck.append(i)
        return spellCheck
def tokenize_text(text):
    tokens=nltk.word_tokenize(text)
    tokens=[token.strip() for token in tokens]
    return tokens
def expand_contractions(text, contraction_mapping):
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),
    flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())
        expanded_contraction = first_char + expanded_contraction[1:]
        return expanded_contraction
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'","", expanded_text)
    return expanded_text
def stemmer_text(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    text = stemmer.stem(text)
    return text
# def lemmatize_text(text):
# pos_tagged_text = pos_tag_text(text)
# lemmatized_tokens = [wnl.lemmatize(word, pos_tag) if pos_tag
# else word for word,
# pos_tag in pos_tagged_text]
# lemmatize_text=' '.join(lemmatized_tokens)
# return lemmatize_text

In [60]:
def remove_special_characters(text):
    tokens = tokenize_text(text)
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    filtered_tokens = filter(None, [pattern.sub('',token) for token in tokens])
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text
factory = StopWordRemoverFactory()
stopword_list = factory.get_stop_words()
def remove_stopwords(text):
    tokens = tokenize_text(text)
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text
def normalize_corpus(corpus, tokenize=False):
    normalized_corpus = []
    for text in corpus:
        text = expand_contractions(text, CONTRACTION_MAP)
        text = stemmer_text(text)
        text = remove_special_characters(text)
        text = repeatcharNormalize(text)
        text = remove_stopwords(text)
        normalized_corpus.append(text)
        if tokenize:
            text = tokenize_text(text)
            text = spellNormalize(text)
            normalized_corpus.append(text)
    return normalized_corpus

In [61]:
dataset = pd.read_csv('cnn_sport.csv')
dataset

,title,category,link
0,Klasemen Liga Inggris: Tottenham Gusur Man Cit...,Sepakbola,https://www.cnnindonesia.com/olahraga/20231024...
1,Khamzat Disebut Tak Pantas Dapat Duel Gelar Ju...,Olahraga Lainnya,https://www.cnnindonesia.com/olahraga/20231023...
2,"Hasil Liga Inggris: Son Cetak Gol, Tottenham H...",Sepakbola,https://www.cnnindonesia.com/olahraga/20231024...
3,Binder Merasa Kasihan Martin Gagal Menang di M...,Moto GP,https://www.cnnindonesia.com/olahraga/20231024...
4,FOTO: Tottenham Jaga Rekor Belum Terkalahkan U...,Sepakbola,https://www.cnnindonesia.com/olahraga/20231024...
...,...,...,...
1995,Jadwal Siaran Langsung Final Hong Kong Open: 3...,Raket,https://www.cnnindonesia.com/olahraga/20230917...
1996,Klasemen Liga Inggris Usai MU Terpuruk di Old ...,Sepakbola,https://www.cnnindonesia.com/olahraga/20230917...
1997,Ronaldo Tunjukkan Gairah Besar di Saudi Pro Le...,Sepakbola,https://www.cnnindonesia.com/olahraga/20230917...
1998,"2 Pemain Al Raed Berebut Tendang Penalti, Rona...",Sepakbola,https://www.cnnindonesia.com/olahraga/20230917...


### Ekstraksi Fitur
Silakan isi bagian ini dengan script untuk keperluan ekstraksi fitur data teks yang akan digunakan untuk pengembangan model klasifikasi teks. 
Untuk setiap strategi ekstraksi fitur yang akan digunakan sebaiknya dibuat fungsi tersendiri dan dipanggil dalam satu fungsi ekstraksi fitur. Jangan lupa pada bagian ini ditampilkan contoh hasil ekstraksi fitur terhadap datanya.

In [62]:
#Fungsi untuk mengekstraksi feature menggunakan TF-IDF Model
def tfidf_transformer(bow_matrix):
    transformer = TfidfTransformer(norm='l2',
                                    smooth_idf=True,
                                    use_idf=True)
    tfidf_matrix = transformer.fit_transform(bow_matrix)
    return transformer, tfidf_matrix
def tfidf_extractor(corpus, ngram_range=(1,1)):
    vectorizer = TfidfVectorizer(min_df=1,
                                norm='l2',
                                smooth_idf=True,
                                use_idf=True,
                                ngram_range=ngram_range)
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features

### Model Training
Silakan isi bagian ini dengan script untuk keperluan training model klasifikasi teks. Jangan lupa untuk menjabarkan rancangan pemodelan algoritme yang akan diterapkan dan kombinasi parameter yang akan digunakan dengan benar dan lengkap.

In [63]:
#training dataset
def prepare_datasets(corpus, labels, test_data_proportion=0.3):
    train_X, test_X, train_Y, test_Y = train_test_split(corpus, labels,
    test_size=0.33,random_state=42)
    return train_X, test_X, train_Y, test_Y
train_corpus, test_corpus, train_labels, test_labels = prepare_datasets(feature,
                                                                        label,test_data_proportion=0.3)

In [66]:
#memanggil fungsi normalisasi dataset yang telah ditraining

norm_train_corpus = normalize_corpus(train_corpus)

In [67]:
norm_train_corpus[0:10]

['latih al nassr ronaldo orang enggak ribet',
 'hasil asi games gregoria kalah badminton indonesia medali',
 'klasemen asi games jumat pagi indonesia rawan salip malaysia',
 'daftar 9 tim negara lolos euro 2024',
 'marquez tinggal honda bye bye honda bagnaia jadi nyata',
 'asi games kuat lentur ramai rumakiek balik gol indah',
 'top skor liga inggris son heung min sejajar salah ancam haaland',
 'jadwal indonesia asi games 2023 jumat 22 september',
 'foto inggris hancur italia tiket euro 2024',
 'messi bangga capai sama inter miami']

In [68]:
norm_test_corpus = normalize_corpus(test_corpus)

In [69]:
norm_test_corpus[0:10]

['hasil liga 1 tangis firza usai cetak gol persija menang dramatis',
 'timnas indonesia bantai brunei 60 laju kualifikasi piala dunia',
 '2 emas asi games sejahtera sama rekor legenda indonesia',
 'neymar umum lahir anak model bruna biancardi',
 '5 nomor peluang medali indonesia asi games hari',
 'latih bongkar alas messi absen inter miami vs houston dynamo',
 'indonesia perak perahu naga 500 meter putri',
 'rionny buka target badminton indonesia asi games 3 emas',
 'pesan penuh makna egy maulana palestina',
 '5 masalah krusial mu wajib baik erik ten hag']